In [1]:
import datetime
import pandas as pd
import numpy as np



In [2]:
from alpha_vantage.timeseries import TimeSeries
alphavantage_api_key = 'LHSF13XCYF6MXI3J'
ts = TimeSeries(key= alphavantage_api_key, output_format='pandas')



## Function: Get Stock Price from AlphaVantage

In [3]:
def get_price_array(ticker):
    try:
        data, meta_data = ts.get_daily(symbol=ticker, outputsize='compact')
        data = data.reset_index()
        data['Ticker'] = ticker
        return data
    except:
        pass

## Load Oil Sentiment Data

In [4]:
csv_path = "Oil_Company_Tweet_Data_011318.csv"
df = pd.read_csv(csv_path, encoding='ISO-8859-1')

df.head()

,Unnamed: 0,Date,Media Sources,Tweet Polarity
0,0,2018-01-03,@MarathonPetroCo,0.23845
1,1,2018-01-04,@DevonEnergy,-0.09200
2,2,2018-01-04,@MarathonPetroCo,0.21230
3,3,2018-01-04,@OPECnews,0.49390
4,4,2018-01-05,@DevonEnergy,-0.27870


In [5]:
df.columns

Index(['Unnamed: 0', 'Date', 'Media Sources', 'Tweet Polarity'], dtype='object')

## Map Media Source to Ticker to later Retrieve Stock Price

In [6]:
oil_ticker = {'@Shell': 'RDS.A ',
             '@OPECnews': 'USO',
              '@exxonmobil': 'XOM',
             '@Chevron': 'CVX',
              '@conocophillips': 'COP',
             '@OXY_Petroleum': 'OXY',
              '@ValeroEnergy': 'VLO',
             '@Phillips66Co': 'PSX',
              '@MarathonPetroCo': 'MPC',
             '@DevonEnergy':'DVN'}

## Get Stock Ticker in order to retrieve Price

In [7]:
df['Ticker'] = [oil_ticker[x] for x in df['Media Sources']]

In [8]:
ticker_list = list(df.Ticker.unique())
ticker_list

['MPC', 'DVN', 'USO', 'OXY', 'VLO', 'PSX', 'COP', 'CVX', 'RDS.A ', 'XOM']

## Generate Stock Prices Chart

In [9]:
stock_df = pd.DataFrame({})
for ticker in ticker_list:
    temp_df = (get_price_array(ticker))
    stock_df = stock_df.append(temp_df, ignore_index=True)


In [10]:
stock_df.tail()

,date,1. open,2. high,3. low,4. close,5. volume,Ticker
895,2018-01-08,86.70,87.15,86.6034,87.14,10549116.0,XOM
896,2018-01-09,86.94,87.15,86.6500,86.77,7870756.0,XOM
897,2018-01-10,87.00,87.19,85.9800,86.08,10624415.0,XOM
898,2018-01-11,86.36,87.37,85.9300,86.93,14358191.0,XOM
899,2018-01-12,87.20,87.99,87.1800,87.52,9833776.0,XOM


In [11]:
stock_df.columns

Index(['date', '1. open', '2. high', '3. low', '4. close', '5. volume',
       'Ticker'],
      dtype='object')

In [12]:
stock_df = stock_df.loc[:,['date', 'Ticker', '4. close']]

In [13]:
stock_df = stock_df.rename(columns={'4. close':'Price'})

In [14]:
stock_df.head()

,date,Ticker,Price
0,2017-08-22,MPC,49.67
1,2017-08-23,MPC,50.17
2,2017-08-24,MPC,51.13
3,2017-08-25,MPC,51.72
4,2017-08-28,MPC,52.52


## To be Deleted

In [ ]:
grouped_df = df.groupby(['Date', 
                        'Tweet Polarity', 
                        'Ticker'])

In [ ]:
mean_df = grouped_df['Tweet Polarity'].mean()

In [ ]:
oil_df = pd.DataFrame(mean_df)
oil_df = oil_df.drop('Tweet Polarity', axis=1)
oil_df = oil_df.reset_index()

In [ ]:
oil_df.head()

In [ ]:
oil_df.tail()

## Get Stock Price for Oil Sentiment (Ticker, Date)

In [23]:
oil_sentiment_df = pd.merge(df, stock_df, left_on=['Date', 'Ticker'], right_on=['date', 'Ticker'], how='left')

In [24]:
oil_sentiment_df.tail()

,Unnamed: 0,Date,Media Sources,Tweet Polarity,Ticker,date,Price
53,53,2018-01-13,@conocophillips,0.133917,COP,NaN,NaN
54,54,2018-01-13,@exxonmobil,0.129404,XOM,NaN,NaN
55,55,2018-01-14,@Chevron,0.000000,CVX,NaN,NaN
56,56,2018-01-14,@Shell,0.000000,RDS.A,NaN,NaN
57,57,2018-01-14,@exxonmobil,0.700300,XOM,NaN,NaN


In [25]:
oil_sentiment_df = oil_sentiment_df.dropna()

In [18]:
oil_sentiment_df.to_csv('oil_sentiment.csv', sep=',')

In [19]:
oil_sentiment_df = oil_sentiment_df.sort_values(by=['Ticker', 'Date'], ascending=[True, True], inplace=False)

In [26]:
oil_sentiment_df.tail()

,Unnamed: 0,Date,Media Sources,Tweet Polarity,Ticker,date,Price
41,41,2018-01-12,@MarathonPetroCo,0.457400,MPC,2018-01-12,71.42
42,42,2018-01-12,@OPECnews,0.399600,USO,2018-01-12,12.87
43,43,2018-01-12,@Phillips66Co,0.395320,PSX,2018-01-12,104.97
44,44,2018-01-12,@ValeroEnergy,0.000000,VLO,2018-01-12,96.75
45,45,2018-01-12,@conocophillips,0.275221,COP,2018-01-12,60.05


In [39]:
oil_sentiment_df.reindex()
oil_sentiment_df.loc[oil_sentiment_df.Ticker =='DVN']

,Unnamed: 0,Date,Media Sources,Tweet Polarity,Ticker,date,Price
1,1,2018-01-04,@DevonEnergy,-0.092000,DVN,2018-01-04,43.23
4,4,2018-01-05,@DevonEnergy,-0.278700,DVN,2018-01-05,43.11
17,17,2018-01-08,@DevonEnergy,0.457400,DVN,2018-01-08,43.25
21,21,2018-01-09,@DevonEnergy,0.537450,DVN,2018-01-09,42.54
27,27,2018-01-10,@DevonEnergy,0.629467,DVN,2018-01-10,42.18
34,34,2018-01-11,@DevonEnergy,0.785671,DVN,2018-01-11,43.07
40,40,2018-01-12,@DevonEnergy,0.605292,DVN,2018-01-12,44.04
